In [2]:
import fxcmpy
TOKEN = "65aa373e2c6d310060a655ac665b46531ecd1adc"
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level="error")
con

In [859]:
import datetime as dt
from icecream import ic 


start = dt.datetime(2021,12,23)
end = dt.datetime(2022,1,7)
data = con.get_candles('EUR/USD', period='m1', start=start, end=end) #number=250 start=start, end=end

In [201]:
import pandas as pd
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1], data['Rates'][2],
             data['Rates'][3]))

def arraySortedOrNot(arr):
 
    # Calculating length
    n = len(arr)
 
    # Array has one or no element or the
    # rest are already checked and approved.
    if n == 1 or n == 0:
        return True
 
    # Recursion applied till last element
    return arr[0] <= arr[1] and arraySortedOrNot(arr[1:])

In [171]:
# import ta
# from ta.trend import *
# from ta.momentum import *


# data["SMALL_SMA"]=sma_indicator(data.bidclose, window=7)
# data["LARGE_SMA"]=sma_indicator(data.bidclose, window=10)
# # Stochastic need to fix
# data["STOCH"] = stoch(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["STOCH_SIGNAL"] = stoch_signal(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["RSI"] = rsi(data.bidclose)
# data["SMA_SIGNAL"] = 

# len(data.index)

# data

In [811]:
import importlib
import strategy
importlib.reload(strategy)
from strategy import BaseStrategy
import pandas as pd
import numpy as np
from ta.trend import *
from ta.momentum import *

class SMAStrategy(BaseStrategy):
    name = "smarsistochastic_strategy"
    description = """7,10 SMA, 7,3,3 Stochastic and RSI Strategy"""
    
    def signal_found(self, data:dict):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "SELL"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "BUY"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        
        df.loc[(df["sma_signal"] == "BUY")  & (df["stoch"] >= 60) & (df["stoch_signal"] >= 60), "stoch_signal_f"] = "CHUTIYA"
        df.loc[(df["sma_signal"] == "SELL")  & (df["stoch"] <= 40) & (df["stoch_signal"] <= 40), "stoch_signal_f"] = "CHUTIYA" 
        df["entry_signal"] = df.sma_signal
        df["entry_price"] = df.bidopen
        return self.signal_found(df.loc[(df["stoch_signal_f"]== "CHUTIYA")].tail(1))
        
    
    def apply_indicator(self):
        super(SMAStrategy, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["stoch"] = round(stoch(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["stoch_signal"] = round(stoch_signal(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 7)


    
# strategy = SMAStrategy()
# # for i in range(51,100):
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# df = strategy.get_signal()
# df = df.loc[(df["SMA_SIGNAL"]=="BUY") | (df["SMA_SIGNAL"]=="SELL")]


# df = df.loc[(df["STOCH_SIGNAL_F"] == "CHUTIYA" )]
# df = df.loc[((df["SMA_SIGNAL"] == "BUY") & (df["SMALL_SMA"] <= df.bidopen) & (df["SMALL_SMA"] <= df.bidclose) &
#            (df["LARGE_SMA"] <= df.bidopen) & (df["LARGE_SMA"] <= df.bidclose)) | 
#            ((df["SMA_SIGNAL"] == "SELL") & (df["SMALL_SMA"] >= df.bidopen) & (df["SMALL_SMA"] >= df.bidclose) &
#            (df["LARGE_SMA"] >= df.bidopen) & (df["LARGE_SMA"] >= df.bidclose))
#            ]
# df = df.loc[((df["SMA_SIGNAL"] == "BUY") & (df["RSI"] >= 70)) | ((df["SMA_SIGNAL"] == "SELL") & (df["RSI"] <= 30))]
# df = df.loc[:, ["bidopen","bidclose","SMA_SIGNAL", "SMALL_SMA", "LARGE_SMA", "RSI", "STOCH","STOCH_SIGNAL"]]
# df.drop(["askopen","askclose", "askhigh", "tickqty"], axis=1)
# df
# df.insert(loc = 0,column = 'Strategy Valid?',value = '<input type="checkbox" />')
# df.tail(50)
# df.style
# df

In [839]:
class SMAStrategy1(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "DOWN_CROSSOVER"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "UP_CROSSOVER"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        # df["entry_signal"] = np.select([(df["sma_signal"] == "UP_CROSSOVER"),(df["sma_signal"] == "DOWN_CROSSOVER")],["BUY","SELL"]) 
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        new_df = df[last_sma_crossover.name:df.iloc[-1].name]
        new_df = new_df.iloc[1: , :]
        signal = None
        data = None
        amt = 0.00003
        if last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            candle_counter = 0
            iter_counter = 0
            signal = "SELL"
            for row in new_df.itertuples():
                if row.max_bid+amt >= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
                # ic(row)
            # sma_touched_df = new_df.head(2).loc[(new_df["max_bid"] >= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "SELL"
            # data = new_df.head(2).tail(1)
        elif last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            candle_counter = 0
            iter_counter = 0
            for row in new_df.itertuples():
                if row.min_bid-amt <= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
            
            # sma_touched_df = new_df.head(2).loc[(new_df["min_bid"] <= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "BUY"
            # data = new_df.head(2).tail(1)
        if signal and not data.empty:
            # ic(signal)
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
        #     # ic(d)
            return d
        # ic(last_sma_crossover)
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        

# strategy = SMAStrategy1()
# strategy.dataframe = data #.iloc[120:187]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).tail(50)
# dd

In [840]:
test_data = data.copy()
import backtest
import strategy
importlib.reload(strategy)
importlib.reload(backtest)
from backtest import BackTestStrategy
importlib.reload(strategy)
importlib.reload(backtest)

sma_backtest = BackTestStrategy(SMAStrategy1, test_data,stoploss=0.0010,target=0.0005)
sma_backtest.start_backtesting()

'backtesting has been completed, please generate report'

In [841]:
import strategy
# importlib.reload(strategy)
kd = sma_backtest.get_report()
# sma_backtest.signal_data
# kd["max_bid"] = kd[["bidopen","bidclose","bidlow","bidhigh"]].max(axis=1)2022-01-03 10:53:00
print(len(list(filter(lambda k : k == "SL", kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", kd.status))) , "Total Target")
kd = kd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
        ], axis=1)
print(kd.shape)
kd

129 Total Stoploss
305 Total Target
(441, 11)


,bidopen,max_bid,min_bid,small_sma,large_sma,sma_signal,entry_price,entry_signal,exit_price,exit_time,status
2021-12-26 23:02:00,1.13187,1.13199,1.13160,1.131321,1.131213,,1.13194,BUY,1.13253,2021-12-26 23:29:00,TG
2021-12-27 00:23:00,1.13197,1.13197,1.13196,1.131833,1.131767,,1.13196,BUY,1.13250,2021-12-27 07:37:00,TG
2021-12-27 00:39:00,1.13195,1.13198,1.13189,1.132023,1.132045,,1.13198,SELL,1.13131,2021-12-27 00:54:00,TG
2021-12-27 01:08:00,1.13202,1.13202,1.13197,1.131863,1.131813,,1.13201,BUY,1.13256,2021-12-27 07:38:00,TG
2021-12-27 01:35:00,1.13193,1.13196,1.13190,1.132087,1.132112,,1.13195,SELL,1.13144,2021-12-27 05:45:00,TG
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-05 21:16:00,1.13099,1.13103,1.13099,1.130906,1.130859,,1.13103,BUY,0.00000,0,0
2022-01-05 22:02:00,1.13107,1.13107,1.13106,1.131157,1.131176,,1.13107,SELL,0.00000,0,0
2022-01-05 22:29:00,1.13088,1.13091,1.13087,1.130964,1.130982,,1.13087,SELL,0.00000,0,0
2022-01-05 23:47:00,1.13105,1.13105,1.13104,1.131154,1.131212,,1.13104,SELL,0.00000,0,0


TypeError: 'map' object is not subscriptable

In [824]:
kd

,bidopen,max_bid,min_bid,small_sma,large_sma,sma_signal,entry_price,entry_signal,exit_price,exit_time,status
2021-12-26 23:02:00,1.13187,1.13199,1.13160,1.131321,1.131213,,1.13194,SELL,1.13145,2021-12-26 23:03:00,TG
2021-12-27 00:00:00,1.13227,1.13227,1.13216,1.132274,1.132289,,1.13216,BUY,1.13176,2021-12-27 00:10:00,SL
2021-12-27 00:23:00,1.13197,1.13197,1.13196,1.131833,1.131767,,1.13196,SELL,1.13155,2021-12-27 00:50:00,TG
2021-12-27 00:39:00,1.13195,1.13198,1.13189,1.132023,1.132045,,1.13198,BUY,1.13158,2021-12-27 00:49:00,SL
2021-12-27 01:08:00,1.13202,1.13202,1.13197,1.131863,1.131813,,1.13201,SELL,1.13160,2021-12-27 02:06:00,TG
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-05 22:29:00,1.13088,1.13091,1.13087,1.130964,1.130982,,1.13087,BUY,1.13132,2022-01-05 23:35:00,TG
2022-01-05 23:02:00,1.13103,1.13103,1.13099,1.130937,1.130915,,1.13100,SELL,0.00000,0,0
2022-01-05 23:19:00,1.13095,1.13097,1.13095,1.130944,1.130919,,1.13095,SELL,1.13136,2022-01-05 23:37:00,SL
2022-01-05 23:46:00,1.13113,1.13114,1.13104,1.131196,1.131244,,1.13104,BUY,0.00000,0,0


In [825]:
data

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2021-12-26 22:29:00,1.13124,1.13073,1.13101,1.13066,1.13223,1.13162,1.13162,1.13162,15
2021-12-26 22:30:00,1.13073,1.13098,1.13131,1.13036,1.13162,1.13180,1.13181,1.13125,20
2021-12-26 22:31:00,1.13098,1.13095,1.13099,1.13095,1.13180,1.13180,1.13180,1.13180,2
2021-12-26 22:32:00,1.13095,1.13116,1.13116,1.13100,1.13180,1.13176,1.13182,1.13173,7
2021-12-26 22:33:00,1.13116,1.13107,1.13119,1.13107,1.13176,1.13177,1.13177,1.13176,4
...,...,...,...,...,...,...,...,...,...
2022-01-05 23:56:00,1.13115,1.13115,1.13116,1.13115,1.13128,1.13127,1.13128,1.13127,4
2022-01-05 23:57:00,1.13114,1.13113,1.13115,1.13112,1.13127,1.13124,1.13127,1.13122,16
2022-01-05 23:58:00,1.13113,1.13106,1.13113,1.13101,1.13123,1.13118,1.13123,1.13113,27


In [363]:
a

[1, 9, 2, 3]